In [46]:
import pickle
import pandas as pd
from cleantext import clean
from nltk.stem import WordNetLemmatizer 
import re
import nltk
import numpy
import random
from gensim.summarization.summarizer import summarize

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag, map_tag
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
words = set(nltk.corpus.words.words())
new_stopwords = set(stopwords.words('english')) - {'not'}

from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")

[nltk_data] Downloading package punkt to /home/mrladidadi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrladidadi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/mrladidadi/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [62]:
with open("mined_sites_0_30.txt", "rb") as fp:   # Unpickling
    all_sites = pickle.load(fp)
with open("mined_sites_filtered_0_30.txt", "rb") as fp2:   # Unpickling
    filt_sites = pickle.load(fp2)
with open("syntactic_analysis/constituency_parse_dict_top.txt", "rb") as fp3:   # Unpickling
    const_parse_lvl1 = pickle.load(fp3)[0]
const_parse_lvl2 = list(pd.read_csv("2_lvl_const_parse.csv")['cons_parse'])
with open("kw_clean.txt", "rb") as fp4:   # Unpickling
    keyword = pickle.load(fp4)

In [49]:
def no_foreign(sent):
    condition = True
    for x in str(sent).split():    
        if not x.isalnum():
            condition = False
    return condition

In [157]:
def candidate(sent, kw):
    if clean(kw, no_punct=True, lang="en") in sent[0:30+len(kw)]:
#     if clean(kw, no_punct=True, lang="en") in sent:
        return True
    else:
        return False

In [158]:
def regex(text):
    clean = re.sub(r'\[[^\]]*\]', '', text)
    clean = re.sub(r'\([^)]*\)', '', clean)
    return clean

In [159]:
def lematize(text,lemat_bool):
    if lemat_bool == True:
        clean_sent = clean_sent.split()
        wl = WordNetLemmatizer()
        clean_sent = [wl.lemmatize(word) for word in clean_sent if not word in new_stopwords]
        clean_sent = ' '.join(clean_sent)
        return clean_sent
    
    else:
        return text

In [160]:
def clean_driver(df, lemat_bool):
    corpus = []
    for site in df:
        temp_arr = []
        
        text = regex(site['text'])
        sent = nltk.sent_tokenize(text) 
        
        for j in range(len(sent)):
            
            clean_sent = clean(sent[j], no_punct=True, lang="en")
            clean_sent = lematize(clean_sent, lemat_bool)
              
            if no_foreign(clean_sent) and candidate(clean_sent, site['kw']) and len(clean_sent.split())<=60:
                temp_arr.append(clean_sent)
        
        corpus.append({'kwn':site['kwn'], 'kw':site['kw'], 'link':site['link'], 'sent':temp_arr} )
          
    return corpus

In [161]:
all_sites_clean = clean_driver(all_sites, False)
filt_sites_clean = clean_driver(filt_sites, False)

In [162]:
def random_filter(master, kwn):
    final = []

    for i in master:
        if i['kwn'] == kwn:
            for j in i['sent']:
                final.append(j)

    if len(final)>=10:
        randarr = []
        for i in range(10):
            randarr.append(final[random.randint(0,len(final)-1)])
        return randarr
    else:
        return final

In [173]:
def tree_lvl1(main):
    main = main['hierplane_tree']['root']['children']
    final = ''
    for i in range(len(main)):
        final += main[i]['nodeType'] + ' '
    return final.strip()

def tree_lvl2(main):
    main = main['hierplane_tree']['root']['children']
    final = ''
    for i in range(len(main)):
        if 'children' in main[i]:
            for j in range(len(main[i]['children'])):
                final += main[i]['children'][j]['nodeType'] + ' '
        else:
            final += '?' + ' '
    return final.strip()


def const_parse_filter(string, lvl_num):
    temp_const = predictor.predict(sentence = string)
    if lvl_num == 1:
        if tree_lvl1(temp_const) in const_parse_lvl1:
            return True
        else:
            return False
    
    if lvl_num == 2:
        if tree_lvl1(temp_const) + ' /// ' + tree_lvl2(temp_const) in const_parse_lvl2:
            return True
        else:
            return False

In [174]:
def const_parse_driver(master, kwn, lvl_num):
    final = []
    count = 0
    for i in master:
        print(int(count/16)*100)
        if i['kwn'] == kwn:
            for j in i['sent']:
                if const_parse_filter(j, lvl_num):
                    final.append(j)
        count += 1
    return final


In [371]:
for i in filt_sites_clean:
    i['pos'] = []
    for j in i['sent']:
        temp_const = predictor.predict(sentence = j)
        i['pos'].append(tree_lvl1(temp_const) + ' /// ' + tree_lvl2(temp_const))

In [372]:
filt_sites_clean[200]

{'kwn': 25,
 'kw': 'regular expression',
 'link': 'https://en.wikipedia.org/wiki/Regular_expression',
 'sent': ['a regular expression is a sequence of characters that define a search pattern',
  'regular expressions are used in search engines search and replace dialogs of word processors and text editors in text processing utilities such as sed and awk and in lexical analysis',
  'each character in a regular expression is either a metacharacter having a special meaning or a regular character that has a literal meaning',
  'a very simple case of a regular expression in this syntax is to locate a word spelled two different ways in a text editor the regular expression serialie matches both serialise and serialize',
  'regular expressions entered popular use from 1968 in two uses pattern matching in a text editor and lexical analysis in a compiler',
  'a regular expression often called a pattern specifies a set of strings required for a particular purpose',
  'the precise syntax for regula

In [394]:
all_sent = []
all_pos = []
for i in filt_sites_clean:
    if i['kwn'] == 17:
        all_sent += i['sent']
        all_pos += i['pos']

FINAL = []

for j in range(len(all_pos)):
    if all_pos[j] in const_parse_lvl2:
        FINAL.append(all_sent[j])
        
print(len(set(FINAL)))
set(FINAL)

6


{'base64 is an encoding scheme used to represent binary data in an ascii format',
 'encodetostring returns the base64 encoding of src',
 'newdecoder constructs a new base64 stream decoder',
 'newencoder returns a new base64 stream encoder',
 'stdencoding is the standard base64 encoding as defined in rfc 4648',
 'urlencoding is the alternate base64 encoding defined in rfc 4648'}

In [348]:
for i in all_sites_clean:
    i['pos2'] = []
    for j in i['pos']:
        temp = ''
        for k in j.split():
            if(k=='///'):
                break
            temp += k + ' '
        i['pos2'].append(temp.strip())

In [370]:
all_sent = []
all_pos = []
for i in all_sites_clean:
    if i['kwn'] == 18:
        all_sent += i['sent']
        all_pos += i['pos2']

FINAL = []

for j in range(len(all_pos)):
    if all_pos[j] in const_parse_lvl1:
        FINAL.append(all_sent[j])
        
print(len(set(FINAL)))
set(FINAL)

7


{'donna mcmaster maintenance mode gives you the option of letting your visitors know that you are working on your website updating or fixing problems',
 'maintenance mode suspends the following featuresrules and monitors rules and monitorsnotifications notificationsautomatic responses automatic responsesstate changes state changesnew alerts new alertsfor example an exchange mailbox role running on a windows server will have an exchange server service pack applied',
 'our maintenance mode is flexible customizable and controlled remotely from your managewp dashboard',
 'thats what maintenance mode does and its why wordpress has it as a builtin feature whenever you update wordpress itself wordpress will put your website into maintenance mode automaticallybut the problem with wordpress builtin maintenance mode is that you cant pull it up whenever you need it',
 'the maintenance mode page appears',
 'this cli maintenance mode is often used during cli version upgrades',
 'users will only see

In [176]:
#must be given entire array to function
def textrank(para, ratio):
    if len(para) >=2:
        temp_para = ''
        for i in para:
            temp_para += i + '. '
        return summarize(temp_para, ratio=ratio, word_count=None, split=True)
    else:
        return para

In [235]:
def textrank_driver(master, kwn, ratio):
    final = []
    count = 0
    for i in master:
        if i['kwn'] == kwn:
            temp = textrank(i['sent'], ratio)
            if len(temp)>=1:
                final += temp
    return final
